# Youtube
>
> YouTube Search ( https://github.com/joetats/youtube_search ) 패키지는 
>
> YouTube의 강력한 속도 제한이 걸린 API를 사용하지 않고 YouTube 동영상을 검색할 수 있는 도구입니다.
>
> 이 패키지는 YouTube 홈페이지의 검색 양식을 활용하고, 검색 결과 페이지를 스크레이핑하여 동영상을 검색합니다.
>
> 이 노트북은 YouTube를 검색하는 도구를 사용하는 방법을 보여줍니다.
>

In [1]:
from langchain_community.tools import YouTubeSearchTool
tool = YouTubeSearchTool()


In [2]:
tool.run("백종원, 1")

"['https://www.youtube.com/shorts/wLVfFj7vINw']"

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
)

In [5]:
tools = [tool]
llm_with_tools = llm.bind_tools(tools)


In [6]:
from langchain import hub

# Fetches the latest version of this prompt
prompt = hub.pull("wfh/langsmith-agent-prompt:5d466cbc")
prompt.pretty_print()

================================ System Message ================================

You are a helpful assistant.

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


In [7]:
from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

runnable_agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

agent_executor = AgentExecutor(
    agent=runnable_agent, tools=tools, handle_parsing_errors=True
)

In [8]:
results = agent_executor.invoke({"input":"MCP에 관련된 유튜브 동영상을 찾아줘"})
results

{'input': 'MCP에 관련된 유튜브 동영상을 찾아줘',
 'output': '다음은 MCP에 관련된 유튜브 동영상입니다:\n\n1. [MCP 동영상 1](https://www.youtube.com/watch?v=fkqXQOjj8cA)\n2. [MCP 동영상 2](https://www.youtube.com/watch?v=Qdu6Sv-NpeU)\n\n관심 있는 내용을 확인해 보세요!'}